In [1]:
!pip install transformers
!pip install tensorflow
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstal

In [2]:
from transformers import AutoTokenizer, TFBertForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import tensorflow as tf
from tensorflow.keras import regularizers
import pandas as pd
import re
from datasets import Dataset

In [2]:
pred_sentences = ["GME green suck it nerds", 'I hate this', 'I love this', 'this is okay', 'you are amazing', 'it doesnt matter']


In [ ]:
gme_df= pd.read_csv("./Crypto_c.csv")

In [ ]:
gme_df

,DATA_COLUMN,LABEL_COLUMN,predicted,train_pred
0,dad works bitcoin told bitcoin missed earnings,0.0,2,0
1,gt bitcoins fixed supply led fans consider aki...,1.0,2,1
2,bitcoin printer goes brrrrrrrr,1.0,0,1
3,twitter enable bitcoin solana xrp crypto payme...,2.0,0,2
4,looks like read bunch apologia shady latin ame...,0.0,0,0
...,...,...,...,...
2989,community buy throw 1000 doge,2.0,1,2
2990,whats objective price domain guys reckon id gi...,1.0,0,1
2991,doge say put 100 last year sake diversity made...,1.0,0,2
2992,yeah like said 62 days ago no buy doge,1.0,1,0


In [ ]:
gme_df = gme_df.dropna(axis= 0)

In [ ]:
gme_df.head()

,DATA_COLUMN,LABEL_COLUMN,predicted
0,dad works bitcoin told bitcoin missed earnings,0.0,2
1,gt bitcoins fixed supply led fans consider aki...,1.0,2
2,bitcoin printer goes brrrrrrrr,1.0,0
3,twitter enable bitcoin solana xrp crypto payme...,2.0,0
4,looks like read bunch apologia shady latin ame...,0.0,0


In [ ]:
print(gme_df.shape)

(2994, 3)


In [ ]:
gme_df["LABEL_COLUMN"].dtype

dtype('float64')

In [ ]:
train = gme_df.iloc[:2500,:]
test = gme_df.iloc[2500:2994, :]

In [ ]:
#train_data = Dataset.from_pandas(train)
#train_token = tokenizer(train_data['DATA_COLUMN'], return_tensors='np', padding=True,)
#train_data = train_data.shuffle(100).batch(32).repeat(2)

#validation_data = Dataset.from_pandas(test)
#validation_data = validation_data.batch(32)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
    return train_InputExamples, validation_InputExamples

    train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                            test, 
                                                                            'DATA_COLUMN', 
                                                                            'LABEL_COLUMN')

def convert_examples_to_tf_dataset(examples, tokenizer, max_length=250):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(500).batch(16).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(16)

In [ ]:
#model.classifier.kernel_regularizer = regularizers.l2(0.01)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
314/314 [==============================] - 309s 832ms/step - loss: 0.9527 - accuracy: 0.5434 - val_loss: 0.7331 - val_accuracy: 0.6822
Epoch 2/2
314/314 [==============================] - 260s 829ms/step - loss: 0.6066 - accuracy: 0.7488 - val_loss: 1.2130 - val_accuracy: 0.5911


In [ ]:
from huggingface_hub import notebook_login



In [ ]:
notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
tokenizer.push_to_hub('crypto-Bert-test')

CommitInfo(commit_url='https://huggingface.co/baz08/crypto-Bert-test/commit/def921bd9351539e1ab25654898ede6e9251770e', commit_message='Upload tokenizer', commit_description='', oid='def921bd9351539e1ab25654898ede6e9251770e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub('crypto-Bert-test')


tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
model.save_pretrained('C:/Users/Barrett/mec-mini-projects-master/api/ML')

In [4]:
model = TFAutoModelForSequenceClassification.from_pretrained("C:/Users/Barrett/mec-mini-projects-master/api/ML/bert-config")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at C:/Users/Barrett/mec-mini-projects-master/api/ML/bert-config were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at C:/Users/Barrett/mec-mini-projects-master/api/ML/bert-config.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without furthe

In [ ]:
pred_labels = []
labels=[0, 1, 2]

index = 0
stride = 100

while index < gme_df.shape[0]:  # Use gme_df.axis[0]:
  last_index = index + stride
  if last_index > 2994:
    last_index = 2994
  pred_sentences = list(gme_df['DATA_COLUMN'])[index : last_index]

  # pred_labels = []
  tf_batch = tokenizer(pred_sentences, max_length = 230, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()

  for i in range(len(pred_sentences)):
    pred_labels.append(labels[label[i]])

  index += stride
  print(index)
  # print(len(pred_sentences))
  # print(len(pred_labels))



print(len(pred_labels))

# for i in range(len(pred_sentences)):
#     print(pred_sentences[i], ": \n", labels[label[i]])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
2994


In [13]:
def get_predictions(model, tokenizer, data):
  labels=['Negative', 'Neutral', 'Positive']
  pred_labels = []
  tf_batch = tokenizer(data, max_length = 230, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  for i in range(len(pred_sentences)):
    print(pred_sentences[i], ": \n", labels[label[i]])
    pred_labels.append(labels[label[i]])

In [1]:
pred_sentences = 'I love bitcoin'

In [16]:
get_predictions(model, tokenizer, pred_sentences)

i love bitcoin : 
 Positive
etc sucks : 
 Negative


In [ ]:
for i in labels:
  print(pred_labels.count(i))

653
1573
768


In [ ]:
gme_df['train_pred'] = pred_labels

In [ ]:
gme_df.to_csv('./Crypto_c.csv', header= True, index = False)

In [ ]:
gme_df.head(10)

,DATA_COLUMN,LABEL_COLUMN,predicted
0,dad works bitcoin told bitcoin missed earnings,0.0,2
1,gt bitcoins fixed supply led fans consider aki...,1.0,2
4,bitcoin printer goes brrrrrrrr,1.0,0
5,twitter enable bitcoin solana xrp crypto payme...,2.0,0
6,looks like read bunch apologia shady latin ame...,0.0,0
7,damn crypto like internet 90s mean 30 years bi...,2.0,1
8,general concepts simple ordinary income capita...,1.0,0
9,top bitcoin,1.0,0
10,around 20 bear stearns lehman brothers collaps...,0.0,0
11,one thing buttcoiners rcc common bitcoin still...,0.0,0


In [10]:

labels=['Negative', 'Neutral', 'Positive']
pred_labels = []
tf_batch = tokenizer(pred_sentences, max_length = 230, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
label = tf.argmax(tf.nn.softmax(tf_outputs[0], axis=-1), axis =1).numpy()[0]


print(pred_sentences, ": \n", labels[label])


I love bitcoin : 
 Positive


In [ ]:
pred_sentences = list(gme_df['DATA_COLUMN'])[:100]
inputs = tokenizer(pred_sentences, max_length = 300,padding=True, truncation=True, return_tensors='tf')
logits = model(**inputs).logits
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

In [ ]:
class_names = ['negative', 'neutral', 'positive']

In [ ]:
gme_df

In [ ]:
from sklearn import metrics
y_true = gme_df['LABEL_COLUMN']
y_pred = gme_df['predicted']
y_true.value_counts()
print(metrics.accuracy_score(y_true, y_pred))
print(metrics.confusion_matrix(y_true, y_pred, labels=[0,
1, 2]))
print(metrics.classification_report(y_true, y_pred, labels=[0,
1, 2]))

0.2541750167000668
[[ 463  141   77]
 [1050  228  145]
 [ 604  216   70]]
              precision    recall  f1-score   support

           0       0.22      0.68      0.33       681
           1       0.39      0.16      0.23      1423
           2       0.24      0.08      0.12       890

    accuracy                           0.25      2994
   macro avg       0.28      0.31      0.23      2994
weighted avg       0.31      0.25      0.22      2994



In [ ]:
y_true = test['LABEL_COLUMN']
y_pred = test['train_pred']
y_true.value_counts()
print(metrics.accuracy_score(y_true, y_pred))
print(metrics.confusion_matrix(y_true, y_pred, labels=[0,
1, 2]))
print(metrics.classification_report(y_true, y_pred, labels=[0,
1, 2]))

0.6619433198380567
[[ 18  17   3]
 [ 43 275  50]
 [ 17  37  34]]
              precision    recall  f1-score   support

           0       0.23      0.47      0.31        38
           1       0.84      0.75      0.79       368
           2       0.39      0.39      0.39        88

    accuracy                           0.66       494
   macro avg       0.49      0.54      0.50       494
weighted avg       0.71      0.66      0.68       494

